In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import spacy
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pickle

In [2]:
#Preprocessing function
def text_preprocessing(document):
    result = []
    document_n = str(document).lower() #Lowering the document
    document_n_ = re.sub('[^a-z0-9]',' ', document_n) #Substitute the non alpha numeric to space 
    for i in nlp_hotel(document_n_):
        if i.is_digit == True or i.is_stop == True:
            pass
        else:
            if i.pos_ == 'SPACE':
                pass
            else:
               result.append(str(i.lemma_))
    return ' '.join(result)

In [5]:
nlp_hotel = spacy.load('en_core_web_md')

In [69]:
vocab = 40000
max_len_for_sen = 50
Embedding = 50

In [9]:
tokeniz = Tokenizer(num_words=vocab)

In [ ]:
tokeniz.fit_on_texts(x.values)
word_index = tokeniz.word_index

In [68]:
def word_sequence(n_most_words, x, max_len):
    tokeniz = Tokenizer(num_words=n_most_words)
    tokeniz.fit_on_texts(x.values)
    sequen = tokeniz.texts_to_sequences(x.values)
    word_index = tokeniz.word_index
    X = pad_sequences(sequen, maxlen=max_len)
    return X, word_index

In [6]:
#setting the path
all_data = os.getcwd() + '//data//new_reviews.csv'

In [10]:
raw_data = pd.read_csv(all_data)

In [11]:
raw_data.columns

Index(['review', 'rating'], dtype='object')

In [12]:
raw_data.rating.value_counts()

1    17277
0     3214
Name: rating, dtype: int64

In [15]:
zero_data = raw_data.loc[raw_data.rating == 0,:]

In [16]:
new_dataset = pd.concat([raw_data,zero_data,zero_data,zero_data,zero_data,zero_data], axis = 0)

In [18]:
new_dataset.reset_index(inplace=True)

In [19]:
new_dataset.rating.value_counts()

0    19284
1    17277
Name: rating, dtype: int64

In [20]:
new_dataset['rating'].dtype

dtype('int64')

In [43]:
Y = new_dataset.rating.values

In [22]:
new_dataset['clean_data'] = new_dataset.review.apply(text_preprocessing)

In [23]:
tokeniz.fit_on_texts(new_dataset.clean_data)

In [25]:
word_ind = tokeniz.word_index

In [26]:
word_ind

{'room': 1,
 'hotel': 2,
 'stay': 3,
 't': 4,
 'n': 5,
 'good': 6,
 'staff': 7,
 'night': 8,
 'great': 9,
 'day': 10,
 'time': 11,
 'service': 12,
 'nice': 13,
 'food': 14,
 'beach': 15,
 'like': 16,
 'resort': 17,
 'go': 18,
 'place': 19,
 'clean': 20,
 'restaurant': 21,
 'location': 22,
 'bed': 23,
 'breakfast': 24,
 'get': 25,
 'pool': 26,
 'check': 27,
 'walk': 28,
 'people': 29,
 'book': 30,
 'small': 31,
 'tell': 32,
 'want': 33,
 'look': 34,
 'area': 35,
 'bad': 36,
 'desk': 37,
 'bar': 38,
 'ask': 39,
 'bathroom': 40,
 'water': 41,
 'well': 42,
 'come': 43,
 'pay': 44,
 'leave': 45,
 'floor': 46,
 'need': 47,
 'minute': 48,
 'take': 49,
 'think': 50,
 'try': 51,
 'little': 52,
 'price': 53,
 'trip': 54,
 'friendly': 55,
 'thing': 56,
 'problem': 57,
 'work': 58,
 'view': 59,
 'say': 60,
 'drink': 61,
 'hour': 62,
 'lot': 63,
 'review': 64,
 'recommend': 65,
 'way': 66,
 'return': 67,
 'experience': 68,
 'arrive': 69,
 'know': 70,
 'star': 71,
 'eat': 72,
 'feel': 73,
 'door': 7

In [70]:
X,word_i  = word_sequence(vocab,new_dataset['clean_data'],max_len_for_sen)

In [71]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3)

In [72]:
x_test.shape

(10969, 50)

In [73]:
y_test.shape

(10969,)

In [87]:
hotel_ = models.Sequential()
hotel_.add(layers.Embedding(vocab,50, input_length=max_len_for_sen))
hotel_.add(layers.LSTM(100, dropout=0.2))
hotel_.add(layers.Dense(80, activation='relu'))
hotel_.add(layers.Dense(1, activation='sigmoid'))
print(hotel_.summary())

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 50, 50)            2000000   
_________________________________________________________________
lstm_45 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dense_38 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 81        
Total params: 2,068,561
Trainable params: 2,068,561
Non-trainable params: 0
_________________________________________________________________
None


In [88]:
hotel_.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [89]:
hotel_.fit(x_train,y_train,batch_size=100, epochs=10)

Train on 25592 samples
Epoch 1/10
25592/25592 [==============================] - 74s 3ms/sample - loss: 0.4097 - acc: 0.8095
Epoch 2/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.1698 - acc: 0.9387
Epoch 3/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.1105 - acc: 0.9622
Epoch 4/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.0667 - acc: 0.9781
Epoch 5/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.0382 - acc: 0.9891
Epoch 6/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.2966 - acc: 0.9413
Epoch 7/10
25592/25592 [==============================] - 52s 2ms/sample - loss: 0.0458 - acc: 0.9877
Epoch 8/10
25592/25592 [==============================] - 53s 2ms/sample - loss: 0.0281 - acc: 0.9931
Epoch 9/10
25592/25592 [==============================] - 53s 2ms/sample - loss: 0.0356 - acc: 0.9880
Epoch 10/10
25592/25592 [==============================] - 

In [90]:
hotel_.evaluate(x_test,y_test)

10969/10969 [==============================] - 25s 2ms/sample - loss: 0.2297 - acc: 0.9410


[0.22969716077702548, 0.9410156]

In [91]:
hotel_.save('hotel_keras_97.h5')

In [95]:
y_pred = hotel_.predict_classes(x_test)

In [96]:
y_pred

array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [47]:
y_pred1[3]

array([2.7349088e-04, 1.0075392e-02, 3.0731128e-03, 9.8443675e-01,
       2.1412314e-03], dtype=float32)

In [57]:
y_pred[13]

2